In [43]:
import torch
import torch.nn as nn
import xml.etree.ElementTree as ET
import numpy as np
import os
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import plotly.io as pio
pio.renderers.default = 'iframe_connected' 
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
import matplotlib.pyplot as plt
init_notebook_mode(connected=True)


In [44]:
# ========== DISPLAY ==========

def plot_level(level, title="Generated Level"):
    level_2d = torch.tensor(level).reshape(15, 50)
    plt.figure(figsize=(12, 3))
    plt.imshow(level_2d, cmap="viridis", aspect="auto")
    plt.colorbar()
    plt.title(title)
    plt.show()


In [45]:
def load_tmx_level(tmx_file_path, map_shape=(1, 15, 50)):
    """
    Loads a level from a .tmx file (in XML format with CSV encoding) into a PyTorch tensor.

    Args:
    - tmx_file_path (str): Path to the .tmx file
    - map_shape (tuple): Shape of the tile map (channels, height, width)

    Returns:
    - torch.Tensor: Tensor of shape (1, 15, 50) or similar based on map_shape
    """
    # Parse the XML file
    tree = ET.parse(tmx_file_path)
    root = tree.getroot()

    # Find the data field in the XML structure
    data_field = root.find(".//layer/data[@encoding='csv']")
    if data_field is None:
        raise ValueError("No CSV-encoded tile data found in this .tmx file.")

    # Get the CSV string of tile values
    csv_data = data_field.text.strip()

    # Convert the CSV string into a list of integers
    tile_values = list(map(int, csv_data.split(',')))

    # Reshape the tile values into the map shape (height, width)
    height, width = map_shape[1], map_shape[2]
    if len(tile_values) != height * width:
        raise ValueError(f"Tile data does not match the expected map size: {height}x{width}.")
    
    # Convert to a numpy array and then to a PyTorch tensor
    level = np.array(tile_values, dtype=np.int32).reshape((height, width))
    level_tensor = torch.tensor(level, dtype=torch.long).unsqueeze(0)  # Add channel dimension

    return level_tensor


In [46]:
def load_all_levels_from_folder(folder_path, map_shape=(1, 15, 50)):
    """
    Loads all levels from a folder containing .tmx files into a list of PyTorch tensors.

    Args:
    - folder_path (str): Path to the folder containing .tmx files
    - map_shape (tuple): Shape of the tile map (channels, height, width)

    Returns:
    - list of torch.Tensor: List of tensors, each representing a level
    """
    # List all .tmx files in the directory
    tmx_files = [f for f in os.listdir(folder_path) if f.endswith('.tmx')]
    
    # List to store all levels
    all_levels = []

    # Iterate through each .tmx file and load it
    for tmx_file in tmx_files:
        tmx_file_path = os.path.join(folder_path, tmx_file)
        try:
            # Load the level data from each .tmx file
            level_tensor = load_tmx_level(tmx_file_path, map_shape)
            all_levels.append(level_tensor)
        except ValueError as e:
            print(f"Skipping {tmx_file}: {e}")

    return all_levels

In [47]:
class Generator(nn.Module):
    def __init__(self, input_dim, output_shape=(1, 15, 50), latent_dim=256):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, latent_dim),
            nn.ReLU(),
            nn.Linear(latent_dim, 128 * 4 * 13),
            nn.ReLU()
        )
        self.deconv = nn.Sequential(
            # Start from 4×13 (after reshaping)
            nn.ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), output_padding=(0, 0)),  # 8×26
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), output_padding=(0, 0)),   # 16×52
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, output_shape[0], kernel_size=(4, 5), padding=(1, 1), stride=(1, 1)),  # Reduced padding to get 15×50            nn.Softmax(dim=1)
        )
    def forward(self, x):
        x = self.fc(x)
        x = x.view(-1, 128, 4, 13)
        x = self.deconv(x)
        return x

In [48]:
class Discriminator(nn.Module):
    def __init__(self, input_channels, cond_dim):
        super().__init__()
        self.level_net = nn.Sequential(
            nn.Conv2d(input_channels, 32, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            nn.Flatten()
        )
        self.cond_net = nn.Sequential(
            nn.Linear(cond_dim, 128),
            nn.LeakyReLU(0.2)
        )
        # Updated with correct flattened size for 15x50 input
        self.fc = nn.Sequential(
            nn.Linear(64 * 3 * 12 + 128, 1),
            nn.Sigmoid()
        )
    def forward(self, x, cond):
        x_feat = self.level_net(x)
        cond_feat = self.cond_net(cond)
        combined = torch.cat([x_feat, cond_feat], dim=1)
        return self.fc(combined)

In [49]:
def train_gan(generator, discriminator, dataloader, num_epochs, device):
    g_opt = torch.optim.Adam(generator.parameters(), lr=2e-4, betas=(0.5, 0.999))
    d_opt = torch.optim.Adam(discriminator.parameters(), lr=2e-4, betas=(0.5, 0.999))
    loss_fn = nn.BCELoss()

    for epoch in range(num_epochs):
        for real_levels, cond_vecs in dataloader:
            real_levels = real_levels.to(device)  # shape: [B, 1, 15, 50]
            cond_vecs = cond_vecs.to(device)

            batch_size = real_levels.size(0)
            real_labels = torch.ones(batch_size, 1).to(device)
            fake_labels = torch.zeros(batch_size, 1).to(device)

            # -----------------------
            # Train Discriminator
            # -----------------------
            z = torch.randn(batch_size, cond_vecs.shape[1]).to(device)
            fake_levels = generator(z)

            d_real = discriminator(real_levels, cond_vecs)
            d_fake = discriminator(fake_levels.detach(), cond_vecs)

            d_loss = loss_fn(d_real, real_labels) + loss_fn(d_fake, fake_labels)

            d_opt.zero_grad()
            d_loss.backward()
            d_opt.step()

            # -----------------------
            # Train Generator
            # -----------------------
            g_fake = discriminator(fake_levels, cond_vecs)
            g_loss = loss_fn(g_fake, real_labels)

            g_opt.zero_grad()
            g_loss.backward()
            g_opt.step()

        print(f"[{epoch+1}/{num_epochs}] D_loss: {d_loss.item():.4f}, G_loss: {g_loss.item():.4f}")


In [50]:
def check_variable_type(variable):
  var_type = type(variable)
  if var_type is int:
    print(f"'{variable}' is an integer (which in Python 3 can represent 'long long' or '__int64').")
  elif var_type is float:
    print(f"'{variable}' is a float.")
  else:
    print(f"'{variable}' is of type: {var_type}")

In [51]:
#loading data
folder_path = "./2D game Project/good/"
levels = load_all_levels_from_folder(folder_path, map_shape=(1, 15, 50))

# Check how many levels were loaded
print(f"Loaded {len(levels)} levels.")


float_levels = [level.float() for level in levels]

stacked_levels = torch.stack(float_levels, dim=0)

levels=stacked_levels


Loaded 1303 levels.


In [52]:
def load_and_test_generator(num_samples=10):
    # Define the same parameters used during training
    input_dim = 100
    output_shape = (1, 15, 50)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Initialize a new generator with the same architecture
    generator = Generator(input_dim=input_dim, output_shape=output_shape).to(device)
    
    # Load the saved state dictionary
    generator.load_state_dict(torch.load('trained_generator.pth'))
    
    # Set to evaluation mode
    generator.eval()
    
    # Generate samples
    with torch.no_grad():
        noise = torch.randn(num_samples, input_dim, device=device)
        generated_levels = generator(noise).cpu()
        generated_levels_int = torch.round(generated_levels).int()
    print(f"Generated {num_samples} new level samples")
    return generated_levels_int

In [53]:
def generate_n_plot():
    generated_samples = load_and_test_generator(num_samples=1)
    array = generated_samples[0].numpy()
    array=np.maximum(array,0)
    flattened_array = array.flatten()
    comma_separated = ', '.join(map(str, flattened_array))
    plot_level(flattened_array)

In [54]:
def clean_n_plot(level):
    array = level[0].numpy()
    array=np.maximum(array,0)
    flattened_array = array.flatten()
    comma_separated = ', '.join(map(str, flattened_array))
    plot_level(flattened_array)

In [ ]:
def train_gan(generator, discriminator, levels, num_epochs=100, batch_size=16, 
              input_dim=100, cond_dim=1, lr=0.0002*0.4, device="cuda"):
    """
    Train the GAN model.
    
    Args:
        generator: The generator model
        discriminator: The discriminator model
        levels: List of real levels with shape [1, 15, 50]
        num_epochs: Number of training epochs
        batch_size: Batch size for training
        input_dim: Dimension of the input noise vector
        cond_dim: Dimension of the conditional input
        lr: Learning rate
        device: Device to train on ('cuda' or 'cpu')
    
    Returns:
        Trained generator and discriminator
    """
    
    # Move models to device
    generator = generator.to(device)
    discriminator = discriminator.to(device)
    
    # Create dataset from levels list
    levels_tensor = torch.stack(levels) if not isinstance(levels, torch.Tensor) else levels
    # Create dummy conditions (modify as needed for your specific conditions)
    dummy_conditions = torch.zeros(len(levels_tensor), cond_dim)
    dataset = TensorDataset(levels_tensor, dummy_conditions)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    # Loss function
    criterion = nn.BCELoss()
    
    # Optimizers
    optimizer_G = optim.Adam(generator.parameters(), lr=lr, betas=(0.5*0.3, 0.999))
    optimizer_D = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))
    
    # Labels for real and fake
    real_label = 1
    fake_label = 0
    
    # Training loop
    g_losses = []
    d_losses = []
    
    for epoch in range(num_epochs):
        for i, (real_levels, conditions) in enumerate(dataloader):
            real_levels = real_levels.to(device)
            conditions = conditions.to(device)
            batch_size = real_levels.size(0)
            
            ############################
            # Train Discriminator
            ############################
            # Real samples
            optimizer_D.zero_grad()
            output_real = discriminator(real_levels, conditions)
            label_real = torch.full((batch_size, 1), real_label, device=device)
            output_real=output_real.float()
            label_real=label_real.float()
            loss_real = criterion(output_real, label_real)
            loss_real.backward()
            
            # Fake samples
            noise = torch.randn(batch_size, input_dim, device=device)
            fake_levels = generator(noise)
            label_fake = torch.full((batch_size, 1), fake_label, device=device)
            output_fake = discriminator(fake_levels.detach(), conditions)
            output_fake=output_fake.float()
            label_fake=label_fake.float()
            loss_fake = criterion(output_fake, label_fake)
            loss_fake.backward()
            
            # Update discriminator
            loss_d = loss_real + loss_fake
            optimizer_D.step()
            
            ############################
            # Train Generator
            ############################
            optimizer_G.zero_grad()
            # Generate fake samples again
            output_fake = discriminator(fake_levels, conditions)
            # Generator wants discriminator to think its outputs are real
            loss_g = criterion(output_fake, label_real)
            loss_g.backward()
            optimizer_G.step()
            
            # Save losses for plotting
            g_losses.append(loss_g.item())
            d_losses.append(loss_d.item())
        
        # Print progress
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}/{num_epochs} | D Loss: {loss_d.item():.4f} | G Loss: {loss_g.item():.4f}")
            # Generate a sample to show progress
            with torch.no_grad():
                sample_noise = torch.randn(1, input_dim, device=device)
                sample_level = generator(sample_noise).cpu()
                sample_level_int = torch.round(sample_level).int()
                # You can add visualization code here if needed
                plot_level(sample_level_int)

    
    return generator, discriminator, g_losses, d_losses

In [56]:
def run_training(num_epochs,batch_size):


    
    # Assuming you have these defined:
    # - levels: list of real levels with shape [1, 15, 50]
    # - input_dim: dimension of the input noise vector (e.g., 100)
    # - Generator and Discriminator classes
    
    # Parameters
    input_dim = 100
    cond_dim = 1
    output_shape = (1, 15, 50)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Initialize models
    generator = Generator(input_dim=input_dim, output_shape=output_shape).to(device)
    discriminator = Discriminator(input_channels=output_shape[0], cond_dim=cond_dim).to(device)
    
    # Train the GAN
    generator, discriminator, g_losses, d_losses = train_gan(
        generator,
        discriminator, 
        levels,
        num_epochs,
        batch_size, 
        input_dim=input_dim,
        cond_dim=cond_dim,
        device=device
    )
    print(len(g_losses), len(d_losses))

  # Create traces
    trace1 = go.Scatter(y=g_losses, mode='lines', name='Generator Loss')
    trace2 = go.Scatter(y=d_losses, mode='lines', name='Discriminator Loss')
    
    layout = go.Layout(title='GAN Training Loss', xaxis=dict(title='Iterations'), yaxis=dict(title='Loss'))
    
    fig = go.Figure(data=[trace1, trace2], layout=layout)
    iplot(fig)

    # Generate and save some samples
    with torch.no_grad():
        noise = torch.randn(10, input_dim, device=device)
        generated_levels = generator(noise).cpu()
        
        # Save the generator model
        torch.save(generator.state_dict(), 'trained_generator.pth')
        
        print("Training complete. Generator saved as 'trained_generator.pth'")
        return generated_levels

In [57]:
num_epochs=100000
batch_size=128

generated_samples = run_training(num_epochs,batch_size)


Epoch 10/100000 | D Loss: 1.4577 | G Loss: 0.7507


TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [ ]:
# input_dim = 100
# generator = Generator(input_dim=input_dim, output_shape=(1, 15, 50))

# latent_vector = torch.randn(1, input_dim)  # Batch size of 1
# generated_level = generator(latent_vector)

# # Check the shape of the generated level
# print("Generated level shape:", generated_level.shape)


# cond_dim_placeholder = 1
# dummy_conditions = torch.zeros(batch_size, cond_dim_placeholder)

# # Pass the dummy data through the discriminator

# discriminator = Discriminator(1, 1)

# output = discriminator(generated_level, dummy_conditions)

# # Print the output shape
# print("Discriminator Output Shape:", output.shape)

# # You can also print the output values to see the predictions
# print("Discriminator Output Values (probabilities):")
# plot_level(output)


# #gan = train_gan(generator, discriminator, dataloader, num_epochs, device)